# Exploring the design space of combinatorial CRISPR experiments

## 1. Load packages

In [1]:
using Random 
using Plots  
using Distributions 
using LinearAlgebra
using Combinatorics
using BioCCP


(process:18520): GLib-GIO-WARNING **: 19:47:54.786: Unexpectedly, UWP app `HaukeGtze.7-ZipFileManagerUnofficial_1.2107.2.0_x64__6bk20wvc8rfx2' (AUMId `HaukeGtze.7-ZipFileManagerUnofficial_6bk20wvc8rfx2!zFM.exe') supports 36 extensions but has no verbs


## 2. Simulation-based approach

### 2.1 gRNA relative frequency distributionin combinatorial gRNA/Cas library

In [2]:
function gRNA_frequency_distribution(m, sd, l, u, gRNA_total; normalize = true, visualize=false)
    d_gRNAlibrary = truncated(Normal(m, sd), l, u)
    gRNA_abundances = collect(rand(d_gRNAlibrary, gRNA_total))
     if visualize
        return histogram(gRNA_abundances, label="", 
            xlabel="Number of reads per gene", 
            ylabel="absolute frequency", title="Read distribution")
    else
        if normalize
            gRNA_abundances /= sum(gRNA_abundances)
        end
        return gRNA_abundances
    end
end

gRNA_frequency_distribution (generic function with 1 method)

### 2.2 gRNA-specific genome editing efficiency

In [3]:
function gRNA_activity_distribution(p_high_activity, μ_high_activity, μ_low_activity, σ_activity, n_gRNAs; visualize=false)   
    d_activity = Binomial(1, p_high_activity)
    d_highactivity = truncated(Normal(μ_high_activity, σ_activity), 0.01, 1)
    d_lowactivity = truncated(Normal(μ_low_activity, σ_activity), 0.01, 1)
    p_gRNA_act = zeros(n_gRNAs) # initialize
    for i in 1:n_gRNAs
        if rand(d_activity, 1) == [1]
            p_gRNA_act[i] = rand(d_highactivity, 1)[1]
        else
            p_gRNA_act[i] = rand(d_lowactivity, 1)[1]
        end
    end
    if visualize
        return histogram(p_gRNA_act, label="", xlabel="gRNA activity", ylabel="absolute frequency", title="gRNA activity distribution")

    else
        return p_gRNA_act
    end
end

gRNA_activity_distribution (generic function with 1 method)

### 2.3 Simulation function

In [4]:
function simulate_Nₓ₁(n_targets, 
                                         n_gRNA_pergene, 
                                         n_gRNA_perconstruct, 
                                         n_gRNA_total, 
                                         p_gRNA_library, 
                                         p_gRNA_act, ϵ_knockout_global; iter=500)
    """ 
    INPUT
  
    
    OUTPUT
    E: expected minimum number of plants to gRNA_read_distributionsee each pairwise combination at least once
    sd: standard deviation on the minimum number of plants
    """
    @assert n_targets * n_gRNA_pergene == n_gRNA_total
    
    T_vec = [] #stores number of plants required for each experiment
        for i in 1:iter       
            genes_vec = [] # Initialize matrix to count pairwise interactions
            T = 0
            while genes_vec != collect(1:n_targets) # check if all pairwise combinations are present
             
                T += 1 # count how many plants must be sampled to fill pairwise interaction matrix
                
                # sample combinatorial gRNA/Cas9 construct
                gRNA_indices_construct = findall((rand(Multinomial(n_gRNA_perconstruct, p_gRNA_library))) .!= 0)
                
                # execute mutations
                gRNA_indices_mutations = [gRNA for gRNA in gRNA_indices_construct if rand(Binomial(1, p_gRNA_act[gRNA])) == 1]
            
                # effective gene knockout (loss-of-function) ?
                gRNA_indices_KO = [gRNA for gRNA in gRNA_indices_mutations if rand(Binomial(1, ϵ_knockout_global)) == 1]
            
                # which genes are knocked out?
                genes_indices_KO = Int.(ceil.(gRNA_indices_KO / n_gRNA_pergene))
                 append!(genes_vec, genes_indices_KO)
                genes_vec = Int.(sort(unique(genes_vec)))
            end
            push!(T_vec, T)   
              
        end
        E = mean(T_vec); sd = std(T_vec)
    return E, sd
end
   

simulate_Nₓ₁ (generic function with 1 method)

In [5]:
using BioCCP

function BioCCP_Nₓ₁(n_targets, 
                                         n_gRNA_pergene, 
                                         n_gRNA_perconstruct, 
                                         n_gRNA_total, 
                                         p_gRNA_library, 
                                         p_gRNA_act, ϵ_knockout_global)
    
    p_gRNAs = p_gRNA_library .* p_gRNA_act * ϵ_knockout_global
    p_genes = [sum(p_gRNAs[i:i+n_gRNA_pergene-1]) for i in 1:n_gRNA_pergene:n_gRNA_total]
    return expectation_minsamplesize(n_targets; p=p_genes, r=n_gRNA_perconstruct, normalize=false), 
    std_minsamplesize(n_targets; p=p_genes, r=n_gRNA_perconstruct, normalize=false)
end

function simulate_Nₓ₂(n_targets, 
                                         n_gRNA_pergene, 
                                         n_gRNA_perconstruct, 
                                         n_gRNA_total, 
                                         p_gRNA_library, 
                                         p_gRNA_act, ϵ_knockout_global; iter=500)
    """ 
    INPUT
  
    
    OUTPUT
    E: expected minimum number of plants to gRNA_read_distributionsee each pairwise combination at least once
    sd: standard deviation on the minimum number of plants
    """
    @assert n_targets * n_gRNA_pergene == n_gRNA_total
#     @assert sum(p_gRNA_library) == 1
    
    T_vec = [] #stores number of plants required for each experiment
        for i in 1:iter     
            if i % 100 == 0
                println("Iteration: $i ... Calculating minimum number of plants ... \n")
            end   
            X_interactions_count = zeros(n_targets, n_targets) # Initialize matrix to count pairwise interactions
            T = 0
            while X_interactions_count != ones(n_targets, n_targets) # check if all pairwise combinations are present
                T += 1 # count how many plants must be sampled to fill pairwise interaction matrix
                
                # sample combinatorial gRNA/Cas9 construct
                gRNA_indices_construct = findall((rand(Multinomial(n_gRNA_perconstruct, p_gRNA_library))) .!= 0)
                
                # execute mutations
                gRNA_indices_mutations = [gRNA for gRNA in gRNA_indices_construct if rand(Binomial(1, p_gRNA_act[gRNA])) == 1]
            
                # effective gene knockout (loss-of-function) ?
                gRNA_indices_KO = [gRNA for gRNA in gRNA_indices_mutations if rand(Binomial(1, ϵ_knockout_global)) == 1]
            
                # which genes are knocked out?
                genes_indices_KO = Int.(ceil.(gRNA_indices_KO / n_gRNA_pergene))
            
                # which pairwise combinations are present?
                interactions = collect(combinations(genes_indices_KO, 2))
                
                # Store represented combinations in matrix
                for interaction in interactions
                    j = interaction[1]; k = interaction[2]
                    X_interactions_count[j,k] = 1; X_interactions_count[k,j] = 1; X_interactions_count[j,j] = 1; X_interactions_count[k,k] = 1          
                end  
            end
            push!(T_vec, T)   
              
        end
        E = mean(T_vec); sd = std(T_vec)
    return E, sd
end

using BioCCP

function BioCCP_Nₓ₂(n_targets, 
                                         n_gRNA_pergene, 
                                         n_gRNA_perconstruct, 
                                         n_gRNA_total, 
                                         p_gRNA_library, 
                                         p_gRNA_act, ϵ_knockout_global)
    
    # how many pairwise combinations of gRNAs
    ind_combinations_gRNA = collect(combinations(1:n_gRNA_total, 2))
    n_combinations_gRNA = length(ind_combinations_gRNA)
    
    # calculate probability and activity of gRNA combinations
    p_combinations_gRNA_library = zeros(n_combinations_gRNA)
    p_combinations_gRNA_act = zeros(n_combinations_gRNA)
    for i in 1:n_combinations_gRNA
        p_combinations_gRNA_library[i] = p_gRNA_library[ind_combinations_gRNA[i][1]] * p_gRNA_library[ind_combinations_gRNA[i][2]]
        p_combinations_gRNA_act[i] = p_gRNA_act[ind_combinations_gRNA[i][1]] * p_gRNA_act[ind_combinations_gRNA[i][2]]
    end
    
    # normalize probability gRNA combinations
    p_combinations_gRNA_library /= sum(p_combinations_gRNA_library)

    # select pairwise gRNA combinations of which each component codes for different gene (goal is to study combinations of knockouts in different genes)
    p_combinations_gRNA_library_interest = []
    p_combinations_gRNA_act_interest = []
    ind_combinations_gRNA_interest = []
    for i in 1:n_combinations_gRNA
        if ceil(ind_combinations_gRNA[i][1]/n_gRNA_pergene) != ceil(ind_combinations_gRNA[i][2]/n_gRNA_pergene)
            push!(p_combinations_gRNA_library_interest, p_combinations_gRNA_library[i])
            push!(p_combinations_gRNA_act_interest, p_combinations_gRNA_act[i])
            push!(ind_combinations_gRNA_interest, ind_combinations_gRNA[i])
        end
    end
        
    n_combinations_gRNA_interest = length(p_combinations_gRNA_library_interest)
    p_combinations_gRNA = p_combinations_gRNA_library_interest .* p_combinations_gRNA_act_interest * ϵ_knockout_global^2

    #### INTEGREREN PER GENCOMBINATIE
    p_genes_matrix = zeros(n_targets, n_targets)
    for i in 1:n_combinations_gRNA_interest
        gene1 = Int(ceil(ind_combinations_gRNA_interest[i][1]/n_gRNA_pergene))
        gene2 = Int(ceil(ind_combinations_gRNA_interest[i][2]/n_gRNA_pergene))
        p_genes_matrix[gene1, gene2] += p_combinations_gRNA[i]
    end
    p_genes = collect([p_genes_matrix[i, j] for j in 2:size(p_genes_matrix, 1) for i in 1:j-1])  
    n_combinations_genes = length(p_genes)
    combinations_pp = length(collect(combinations(1:n_gRNA_perconstruct, 2)))
    
    return expectation_minsamplesize(n_combinations_genes; p=p_genes, r=combinations_pp, normalize=false), std_minsamplesize(n_combinations_genes; p=p_genes, r=combinations_pp, normalize=false)
end
   

BioCCP_Nₓ₂ (generic function with 1 method)

In [6]:
function simulate_Nₓ₃(n_targets, 
                                         n_gRNA_pergene, 
                                         n_gRNA_perconstruct, 
                                         n_gRNA_total, 
                                         p_gRNA_library, 
                                         p_gRNA_act, ϵ_knockout_global; iter=500)
    """ 
    INPUT
  
    
    OUTPUT
    E: expected minimum number of plants to gRNA_read_distributionsee each pairwise combination at least once
    sd: standard deviation on the minimum number of plants
    """
    @assert n_targets * n_gRNA_pergene == n_gRNA_total
#     @assert sum(p_gRNA_library) == 1
    
    T_vec = [] #stores number of plants required for each experiment
        for i in 1:iter     
            if i % 100 == 0
                println("Iteration: $i ... Calculating minimum number of plants ... \n")
            end   
            X_interactions_count = zeros(n_targets, n_targets, n_targets) # Initialize matrix to count triple interactions
            T = 0
            while X_interactions_count != ones(n_targets, n_targets, n_targets) # check if all triple combinations are present
                T += 1 # count how many plants must be sampled to fill triple interaction matrix
                
                # sample combinatorial gRNA/Cas9 construct
                gRNA_indices_construct = findall((rand(Multinomial(n_gRNA_perconstruct, p_gRNA_library))) .!= 0)
                
                # execute mutations
                gRNA_indices_mutations = [gRNA for gRNA in gRNA_indices_construct if rand(Binomial(1, p_gRNA_act[gRNA])) == 1]
            
                # effective gene knockout (loss-of-function) ?
                gRNA_indices_KO = [gRNA for gRNA in gRNA_indices_mutations if rand(Binomial(1, ϵ_knockout_global)) == 1]
            
                # which genes are knocked out?
                genes_indices_KO = Int.(ceil.(gRNA_indices_KO / n_gRNA_pergene))
            
                # which triple combinations are present?
                interactions = collect(combinations(genes_indices_KO, 3))
                
                # Store represented combinations in matrix
                for interaction in interactions
                    j = interaction[1]
                    k = interaction[2]
                    l = interaction[3]
                    X_interactions_count[j,k,l] = 1
                    X_interactions_count[k,j,l] = 1
                    X_interactions_count[l,j,k] = 1
                    X_interactions_count[l,k,j] = 1
                    X_interactions_count[j,l,k] = 1
                    X_interactions_count[k,l,j] = 1
        
                    X_interactions_count[:,l,l] .= 1
                    X_interactions_count[:,k,k] .= 1
                    X_interactions_count[:,j,j] .= 1
                    X_interactions_count[l,:,l] .= 1
                    X_interactions_count[k,:,k] .= 1
                    X_interactions_count[j,:,j] .= 1
        
                    X_interactions_count[j,j,:] .= 1
                    X_interactions_count[k,k,:] .= 1
                    X_interactions_count[l,l,:] .= 1
                end  
            end
            push!(T_vec, T)   
              
        end
        E = mean(T_vec); sd = std(T_vec)
    return E, sd
end
   
        
function BioCCP_Nₓ₃(n_targets, 
                                         n_gRNA_pergene, 
                                         n_gRNA_perconstruct, 
                                         n_gRNA_total, 
                                         p_gRNA_library, 
                                         p_gRNA_act, ϵ_knockout_global)
    
    # how many pairwise combinations of gRNAs
    ind_combinations_gRNA = collect(combinations(1:n_gRNA_total, 3))
    n_combinations_gRNA = length(ind_combinations_gRNA)
    
    # calculate probability and activity of triple gRNA combinations
    p_combinations_gRNA_library = zeros(n_combinations_gRNA)
    p_combinations_gRNA_act = zeros(n_combinations_gRNA)
    for i in 1:n_combinations_gRNA
        p_combinations_gRNA_library[i] = p_gRNA_library[ind_combinations_gRNA[i][1]] * p_gRNA_library[ind_combinations_gRNA[i][2]] * p_gRNA_library[ind_combinations_gRNA[i][3]]
        p_combinations_gRNA_act[i] = p_gRNA_act[ind_combinations_gRNA[i][1]] * p_gRNA_act[ind_combinations_gRNA[i][2]] * p_gRNA_act[ind_combinations_gRNA[i][3]]
    end
    
    # normalize probability gRNA combinations
    p_combinations_gRNA_library /= sum(p_combinations_gRNA_library)

    # select triple gRNA combinations of which each component codes for different gene (goal is to study combinations of knockouts in different genes)
    p_combinations_gRNA_library_interest = []
    p_combinations_gRNA_act_interest = []
    ind_combinations_gRNA_interest = []
    for i in 1:n_combinations_gRNA
        if ceil(ind_combinations_gRNA[i][1]/n_gRNA_pergene) != ceil(ind_combinations_gRNA[i][2]/n_gRNA_pergene) && ceil(ind_combinations_gRNA[i][1]/n_gRNA_pergene) != ceil(ind_combinations_gRNA[i][3]/n_gRNA_pergene) && ceil(ind_combinations_gRNA[i][3]/n_gRNA_pergene) != ceil(ind_combinations_gRNA[i][2]/n_gRNA_pergene)
            push!(p_combinations_gRNA_library_interest, p_combinations_gRNA_library[i])
            push!(p_combinations_gRNA_act_interest, p_combinations_gRNA_act[i])
            push!(ind_combinations_gRNA_interest, ind_combinations_gRNA[i])
        end
    end
        
    n_combinations_gRNA_interest = length(p_combinations_gRNA_library_interest)
    p_combinations_gRNA = p_combinations_gRNA_library_interest .* p_combinations_gRNA_act_interest * ϵ_knockout_global^3

    #### INTEGREREN PER GENCOMBINATIE
    p_genes_matrix = zeros(n_targets, n_targets, n_targets)
    for i in 1:n_combinations_gRNA_interest
        gene1 = Int(ceil(ind_combinations_gRNA_interest[i][1]/n_gRNA_pergene))
        gene2 = Int(ceil(ind_combinations_gRNA_interest[i][2]/n_gRNA_pergene))
        gene3 = Int(ceil(ind_combinations_gRNA_interest[i][3]/n_gRNA_pergene))
        p_genes_matrix[gene1, gene2, gene3] += p_combinations_gRNA[i]
    end
    
    combinations_genes = collect(combinations(1:n_targets, 3))
    p_genes = []
        for combination in combinations_genes
            push!(p_genes, p_genes_matrix[combination[1], combination[2], combination[3]])
        end
        
    n_combinations_genes = length(p_genes)
    combinations_pp = length(collect(combinations(1:n_gRNA_perconstruct, 3)))
    
    return expectation_minsamplesize(n_combinations_genes; p=p_genes, r=combinations_pp, normalize=false), std_minsamplesize(n_combinations_genes; p=p_genes, r=combinations_pp, normalize=false)
end

BioCCP_Nₓ₃ (generic function with 1 method)

In [7]:
n_targets = 20
n_gRNA_pergene = 6
n_gRNA_total = n_targets * n_gRNA_pergene
ϵ_knockout_global = 0.8
ρ = 2
l = 50; 
u = ρ*l
m= (l+u)/2; 
sd= (u-l)/2; 
Random.seed!(1)
p_gRNA_library = gRNA_frequency_distribution(m, sd, l, u, n_gRNA_total; normalize=true)


Random.seed!(1)
p_active = 0.9; high_activity = 0.9; low_activity = 0.1; sd_act = 0.01
p_gRNA_act = gRNA_activity_distribution(p_active, high_activity, low_activity, sd_act, n_gRNA_total; visualize=false);

In [8]:
n_gRNA_perconstruct = 1
Random.seed!(1)
@time simulate_Nₓ₁(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global; iter=500)

  1.339814 seconds (3.42 M allocations: 320.668 MiB, 6.84% gc time, 71.29% compilation time)


(107.798, 35.021567875137535)

In [9]:
Random.seed!(1)
@time BioCCP_Nₓ₁(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global)

  0.735133 seconds (2.54 M allocations: 128.287 MiB, 5.87% gc time, 99.76% compilation time)


(110, 38)

In [10]:
n_gRNA_perconstruct = 2
Random.seed!(1)
@time simulate_Nₓ₂(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global; iter=500)

Iteration: 100 ... Calculating minimum number of plants ... 

Iteration: 200 ... Calculating minimum number of plants ... 

Iteration: 300 ... Calculating minimum number of plants ... 

Iteration: 400 ... Calculating minimum number of plants ... 

Iteration: 500 ... Calculating minimum number of plants ... 

 10.638747 seconds (22.79 M allocations: 6.973 GiB, 12.70% gc time, 4.13% compilation time)


(2785.95, 653.5562769297134)

In [11]:
Random.seed!(1)
@time BioCCP_Nₓ₂(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global)

  1.285499 seconds (1.72 M allocations: 113.445 MiB, 3.53% gc time, 97.63% compilation time)


(2731, 641)

In [12]:
n_gRNA_perconstruct = 3
Random.seed!(1)
@time simulate_Nₓ₃(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global; iter=500)

Iteration: 100 ... Calculating minimum number of plants ... 

Iteration: 200 ... Calculating minimum number of plants ... 

Iteration: 300 ... Calculating minimum number of plants ... 

Iteration: 400 ... Calculating minimum number of plants ... 

Iteration: 500 ... Calculating minimum number of plants ... 

411.362045 seconds (315.17 M allocations: 1.141 TiB, 14.00% gc time, 0.11% compilation time)


(37961.906, 7183.905227227848)

In [13]:
n_gRNA_perconstruct = 3
Random.seed!(1)
@time BioCCP_Nₓ₃(n_targets, 
                                n_gRNA_pergene, 
                                n_gRNA_perconstruct, 
                                n_gRNA_total, 
                                p_gRNA_library, 
                                p_gRNA_act, ϵ_knockout_global)

  3.184293 seconds (69.43 M allocations: 1.176 GiB, 5.36% gc time, 27.13% compilation time)


(36771, 6989)